<a href="https://colab.research.google.com/github/snvssk/data298A/blob/development/SolarEnergy/MLModels/Solar_SVR_Ritanjali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import seaborn as sns
from glob import glob
import numpy as np
from datetime import datetime
from random import randint
from math import sqrt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/all_cities_transformed_combined.csv')
df.columns

Index(['Unnamed: 0', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature',
       'Cloud_Type', 'Dew_Point', 'GHI', 'Relative_Humidity',
       'Solar_Zenith_Angle', 'Surface_Albedo', 'Pressure',
       'Precipitable_Water', 'Wind_Direction', 'Wind_Speed', 'Location'],
      dtype='object')

In [ ]:
# nsrdb_na = df.drop(columns='Unnamed: 0')
nsrdb_na = df.copy()
#null_rows = nsrdb_na[nsrdb_na.isna().any(axis=1)]
#null_rows
nsrdb_na.isnull().sum()
#nsrdb_na.columns
#nsrdb_na['GHI'].isnull().values.sum()
nsrdb = nsrdb_na.dropna()
#print(nsrdb)
nsrdb.isnull().sum()

Unnamed: 0            0
Year                  0
Month                 0
Day                   0
Hour                  0
Minute                0
Temperature           0
Cloud_Type            0
Dew_Point             0
GHI                   0
Relative_Humidity     0
Solar_Zenith_Angle    0
Surface_Albedo        0
Pressure              0
Precipitable_Water    0
Wind_Direction        0
Wind_Speed            0
Location              0
dtype: int64

In [ ]:
nsrdb = nsrdb.drop(['Unnamed: 0'],axis =1)
nsrdb = nsrdb.loc[(nsrdb['Hour'].isin([7, 10, 13, 16, 19])) & (nsrdb['Minute'] == 0)]
nsrdb


,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,GHI,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,0.0,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,290.0,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,421.0,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,114.0,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,0.0,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,0.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,281.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,411.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,84.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


In [ ]:
target = nsrdb['GHI']#keeping only target(ghi) data here
input= nsrdb.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Minute,Temperature,Cloud_Type,Dew_Point,Relative_Humidity,Solar_Zenith_Angle,Surface_Albedo,Pressure,Precipitable_Water,Wind_Direction,Wind_Speed,Location
14,2018.0,1.0,1.0,7.0,0.0,9.8,7.0,8.8,93.25,94.85,0.100,1025.0,1.800,325.0,0.7,117987.0
20,2018.0,1.0,1.0,10.0,0.0,14.0,7.0,9.5,74.48,67.81,0.100,1026.0,1.800,24.0,1.5,117987.0
26,2018.0,1.0,1.0,13.0,0.0,17.6,7.0,8.6,55.38,61.32,0.100,1024.0,1.800,15.0,0.9,117987.0
32,2018.0,1.0,1.0,16.0,0.0,15.4,7.0,9.2,66.62,80.55,0.100,1023.0,1.800,14.0,0.8,117987.0
38,2018.0,1.0,1.0,19.0,0.0,12.8,7.0,8.3,74.04,112.83,0.100,1024.0,1.700,72.0,1.0,117987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946152,2013.0,12.0,31.0,7.0,0.0,9.0,7.0,8.0,100.00,94.75,0.111,1020.0,1.233,174.6,0.4,117546.0
946158,2013.0,12.0,31.0,10.0,0.0,13.0,3.0,7.0,74.38,67.81,0.111,1020.0,1.403,23.4,0.7,117546.0
946164,2013.0,12.0,31.0,13.0,0.0,17.0,7.0,4.0,49.14,61.44,0.111,1020.0,1.334,33.3,1.1,117546.0
946170,2013.0,12.0,31.0,16.0,0.0,7.0,4.0,1.0,73.84,80.77,0.113,1020.0,0.999,343.0,0.8,117546.0


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_normalized = sc.fit_transform(input)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
x_train, x_test, y_train, y_test = train_test_split(input_normalized, target, test_size=0.3, random_state=42)

print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)
print("y_train shape",y_train.shape)
print("y_text shape",y_test.shape)

x_train shape (68985, 16)
x_test shape (29565, 16)
y_train shape (68985,)
y_text shape (29565,)


In [ ]:
regressor = SVR(C= 100, gamma=0.1, kernel= 'rbf')

In [ ]:
#class sklearn.svm.SVR(*, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1

In [ ]:
regressor.fit(x_train, y_train)

SVR(C=100, gamma=0.1)

In [12]:
y_pred = regressor.predict(x_test)
y_pred

array([-3.96804046e-01,  7.08701482e+02, -5.63760922e+00, ...,
        4.27974053e+02,  2.41281365e+02,  6.07501797e-01])

In [ ]:
# save the model to disk
import pickle
filename = 'SVR-3Points-Solar'
pickle.dump(regressor, open(filename, 'wb'))


In [13]:
from sklearn.metrics import r2_score 
R_square = r2_score(y_test, y_pred) 
print('Coefficient of Determination R2', R_square)
from sklearn import metrics

print("Model MAE", metrics.mean_absolute_error(y_test, y_pred))
print("Model MSE", metrics.mean_squared_error(y_test, y_pred))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Coefficient of Determination R2 0.9728798493432839
Model MAE 23.81735438537495
Model MSE 2822.874042544986
Model RMSE 53.130725974194874


# Cupertino Test

In [15]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/cupertino_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=regressor.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 30.263417905386802
Model MSE 2791.7360033304685
Model RMSE 52.836881090110424


# Fostercity Test

In [16]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/fostercity_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
fostercity_final = df.dropna()
fostercity_final.isnull().sum()

target_test = fostercity_final['GHI']#keeping only target(ghi) data here
input_test= fostercity_final.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=regressor.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 31.319770061779778
Model MSE 3177.6959735944147
Model RMSE 56.371056168874595


# Los Gatos Test

In [18]:
import pandas as pd
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/losgatos_test.csv')
df.head()

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=regressor.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 29.117877416259443
Model MSE 2477.938271579375
Model RMSE 49.7788938364381


## Saratoga

In [19]:
df =pd.read_csv('/content/drive/Shareddrives/DATA298/WeatherData/FinalWeatherData/saratoga_2020.csv')
df

df = df.loc[(df['Hour'].isin([7, 10, 13, 16, 19])) & (df['Minute'] == 0)]
#df = df.drop(['Unnamed: 0'],axis =1)
null = df[df.isna().any(axis=1)]
df = df.dropna()
df.isnull().sum()

target_test = df['GHI']#keeping only target(ghi) data here
input_test= df.drop(['GHI'], axis=1)
input_test.head()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
input_nor_test = sc.fit_transform(input_test)
y_pred_test=regressor.predict(input_nor_test)
y_pred_test

from sklearn import metrics
print("Model MAE", metrics.mean_absolute_error(target_test, y_pred_test))
print("Model MSE", metrics.mean_squared_error(target_test, y_pred_test))
print("Model RMSE",np.sqrt(metrics.mean_squared_error(target_test, y_pred_test)))

Model MAE 32.579849166944705
Model MSE 2541.8706290128175
Model RMSE 50.41696766975199
